In [11]:
from fixed_effects import run_fixed_effects_on_flattened
from random_effects import run_rand_effects_on_flattened
import pandas as pd

In [12]:
path = "./../../data/11_flattenned/flatten_variables_pct.csv"

Simple Fixed Effects

In [13]:
results_simple_fix = run_fixed_effects_on_flattened(path)
results_end_of_year_fixed = run_fixed_effects_on_flattened(path,
                                  dummy_dates = ["=12/2019", "=12/2020", "=12/2021", "=12/2022", "=12/2023"], 
                                  dummy_names = ["is_END19", "is_END20", "is_END21", "is_END22", "is_END23"])
results_war_fixed = run_fixed_effects_on_flattened(path,
                                  dummy_dates=[">02/2022"], 
                                  dummy_names=["is_WAR"])
results_both_fixed = run_fixed_effects_on_flattened(path,
                                  dummy_dates = ["=12/2019", "=12/2020", "=12/2021", "=12/2022", "=12/2023", ">02/2022"], 
                                  dummy_names = ["is_END19", "is_END20", "is_END21", "is_END22", "is_END23", "is_WAR"])
results_random = run_rand_effects_on_flattened(path)

lst = [(results_simple_fix, "Simp__F"), 
       (results_end_of_year_fixed, "Years_F"),
       (results_war_fixed, "War_F"),
       (results_both_fixed, "All_F"),
       (results_random, "Random")]

=============================== Fixed Effects =================================
                            OLS Regression Results                            
Dep. Variable:                    NIM   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     3.588
Date:                Thu, 02 May 2024   Prob (F-statistic):           3.28e-07
Time:                        17:29:37   Log-Likelihood:                 301.22
No. Observations:                 550   AIC:                            -560.4
Df Residuals:                     529   BIC:                            -469.9
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [14]:
dct = {"Method":[], "MAE":[], "MAPE":[], "MSE":[], "RMSE":[]}

for i in lst:
    r = i[0].get_estimators(silence=True)
    dct["Method"].append(i[1])
    dct["MAE"].append(r[0])
    dct["MAPE"].append(r[1])
    dct["MSE"].append(r[2])
    dct["RMSE"].append(r[3])

results = pd.DataFrame(dct)
results

,Method,MAE,MAPE,MSE,RMSE
0,Simp__F,0.090333,4.500181,0.019581,0.139931
1,Years_F,0.090672,5.243962,0.019372,0.139182
2,War_F,0.090286,4.203689,0.019572,0.139900
3,All_F,0.090688,4.909646,0.019359,0.139138
4,Random,0.091126,1.973881,0.019961,0.141283


In [15]:
# - - - F-TEST - - -
# f_h = "is_END19 = 0, is_END20 = 0, is_END21 = 0, is_END22 = 0, is_END23 = 0"
# results_end_of_year_fixed.res.f_test(f_h)